#Import library


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

#Read Dataset

In [2]:
fake=pd.read_csv("/content/drive/MyDrive/Research Dataset/Fake News English dataset/Fake.csv")
true=pd.read_csv("/content/drive/MyDrive/Research Dataset/Fake News English dataset/True.csv")

FileNotFoundError: ignored

In [3]:
fake.head()

NameError: ignored

In [ ]:
true.head()

In [ ]:
true['target']=1
fake['target']=0

In [ ]:
df=pd.concat([true,fake])

In [ ]:
df.shape

In [ ]:
df.head(5)

#Target feature View

In [ ]:
temp = df.groupby('target').count()['text'].reset_index().sort_values(by='text',ascending=False)
temp.style.background_gradient(cmap='Purples')

In [ ]:
sns.countplot(x='target',data=df,palette=['orange','purple'])

In [ ]:
temp

In [ ]:
from plotly import graph_objs as go
fig = go.Figure(go.Funnelarea(
    text =temp.target,
    values = temp.text,
    title = {"position": "top center", "text": "Funnel-Chart of Target Distribution"}
    ))
fig.show()

In the final dataset the count of fake news is greater than real news

In [ ]:
df.isnull().sum()

No null values are present in the dataset

In [ ]:
df.nunique()

In [ ]:
df.dtypes

#Subject and Target count

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize=(15,5))
sns.countplot(x='subject',data=df,hue='target')

This shows that all the real news have the subject politicsNews and worldnews. Apart from these subjects other six subjects lies in the fake news category

In [ ]:
df['text']=df['text']+" "+df['title']

In [ ]:
df.drop(['title','subject','date'],axis=1,inplace=True)
df.head()

#Number of words in real anad fake news

In [ ]:
no_words=df[df['target']==1].text.str.split().map(lambda x:len(x))
no_words.plot(kind='hist',edgecolor='black',color='lightgreen',title='no of words in Real')
plt.show()
no_words=df[df['target']==0].text.str.split().map(lambda x:len(x))
no_words.plot(kind='hist',edgecolor='black',color='lightblue',title='no of words in fake')
plt.show()

No of words in real news are lying in the range of 0 to 1500 whereas in the case of fake news it lies in the range of 0 to 2000.This shows that number of words in the fake news are higher than that of real news

#Average length of word in real news and Fake News

In [ ]:
avg_len_word=df[df['target']==1].text.str.split().map(lambda x:np.mean([len(word) for word in x]))
avg_len_word.plot(kind='hist',edgecolor='black',color='lightcoral',title='avg length of words in true')
plt.show()
avg_len_word=df[df['target']==0].text.str.split().map(lambda x:np.mean([len(word) for word in x]))
avg_len_word.plot(kind='hist',edgecolor='black',color='lightyellow',title='avg length of words in false')
plt.show()

The average length of word in real news ranging from 4.5 to 6.0 whereas in the case of fake news it lies in the range of 0 to 20.This shows that in the fake news most of the words have length greater than the words of real news.

#Analyzing the Top StopWords in Real and Fake News

##First way

In [ ]:
# creating sample words
def create_words(target):
    words = []
    for x in df[df['target']==target]['text'].str.split():
        for i in x:
            words.append(i)
    return words

In [ ]:
from collections import defaultdict
def analyze_stopwords(data,fun,target):
    values_list=[]
    dic=defaultdict(int)
    for i in range(0,len(target)):
        corpus=fun(target[i])
        for word in corpus:
            dic[word]+=1
        top=sorted(dic.items(),key=lambda x:x[1],reverse=True)[:10]
        x_items,y_items=zip(*top)
        values_list.append(x_items)
        values_list.append(y_items)
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
    ax1.bar(values_list[0],values_list[1],color="lightblue",edgecolor='black', linewidth=1.2)
    ax1.set_title("Real")
    
    ax2.bar(values_list[2],values_list[3],color="lightgreen",edgecolor='black', linewidth=1.2)
    ax2.set_title("Fake")
            
    plt.suptitle("Top Stop words in text")
    plt.show()

In [ ]:
analyze_stopwords(df,create_words,[1,0])

##Second Way

In [ ]:
from collections import Counter
df['temp_list']=df['text'].apply(lambda x: str(x).split())
top=Counter([word for li in df['temp_list'] for word in li])
temp_1=pd.DataFrame(top.most_common(20))
temp_1.columns=["most_common_words","frequency"]
temp_1.style.background_gradient(cmap='Blues')

In [ ]:
import plotly.express as pe
import plotly.figure_factory as ff
fig = pe.bar(temp_1, x="frequency", y="most_common_words", title='Commmon Words in Text', orientation='h', 
             width=700, height=700,color='most_common_words')
fig.show()

#Analyzing the Top Punctuation Marks in Real and Fake News

In [ ]:
import string
punctuation_list=list(string.punctuation)
value_list=[]
def most_occuring(dataset,fun,target):
    d=defaultdict(int)
    for j in range(0,len(target)):
        words=fun(target[j])
        for i in words:
            if i in punctuation_list:
                d[i]+=1
        top=sorted(d.items(),key=lambda x: x[1],reverse=True)[:10]
        x_items,y_counts=zip(*top)
        value_list.append(x_items)
        value_list.append(y_counts)
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
    ax1.bar(value_list[0],value_list[1],color="lightcoral",edgecolor='black', linewidth=1.2)
    ax1.set_title("Real")
    
    ax2.bar(value_list[2],value_list[3],color="purple",edgecolor='black', linewidth=1.2)
    ax2.set_title("Fake")
            
    plt.suptitle("Punctuations in text")
    plt.show()


In [ ]:
most_occuring(df,create_words,[1,0])

#Text Preprocessing

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')
import nltk
nltk.download('punkt')

In [ ]:
nltk.download('wordnet')

In [ ]:
import re
import string

from nltk.corpus import stopwords

def clean_text(text):
    """Process text function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet
    """
    lemmatizer = WordNetLemmatizer()
    stopwords_english = stopwords.words('english')
    text= re.sub('\[[^]]*\]', '', text)
    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)
    #removal of html tags
    review =re.sub(r'<.*?>',' ',text) 
    # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)
    text = re.sub("["
                           u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+",' ',text)
    text = re.sub('[^a-zA-Z]',' ',text) 
    text = text.lower()
    text_tokens =word_tokenize(text)

    text_clean = []
    for word in  text_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            lem_word =lemmatizer.lemmatize(word)  # lemmitiging word
            text_clean.append(lem_word)
    text_mod=[i for i in text_clean if len(i)>2]
    text_clean=' '.join(text_mod)
    return  text_clean

In [ ]:
df.head()

In [ ]:
df['clean_text']=df['text'].apply(lambda x: clean_text(x))

# df['clean_text']=df['text']

In [ ]:
df['clean_text'][:5]

#Look at the Top words of the dataset

In [ ]:
# wordcloud
from PIL import Image
book_mask = np.array(Image.open('/content/drive/MyDrive/Research Dataset/Mask for world cloud/book-logo-1.jpg'))

wc = WordCloud(
    background_color='white', 
    max_words=200, 
    mask=book_mask,
)
wc.generate(' '.join(text for text in df.loc[:, 'text']))
plt.figure(figsize=(18,10))
plt.title('Top words in the dataset', 
          fontdict={'size': 28,  'verticalalignment': 'bottom'})
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
df['clean_temp']=df['clean_text'].apply(lambda x: str(x).split())
top=Counter([word for li in df['clean_temp'] for word in li])
temp_2=pd.DataFrame(top.most_common(20))
temp_2.columns=["common_words",'frequency']
temp_2.style.background_gradient(cmap='Blues')

In [ ]:
top.most_common(20)

#Top word in True and Fake News

In [ ]:
# analyze top 50 words in Real and false texts
data_1=df[df['target']==1]
pd.Series(' '.join([i for i in data_1.clean_text]).split()).value_counts()[:200].plot(kind='bar',figsize=(20,8),color='yellow'
                                                                                       ,edgecolor='black',title='Real')
plt.show()

data_0=df[df['target']==0]
pd.Series(' '.join([i for i in data_0.clean_text]).split()).value_counts()[:20].plot(kind='bar',figsize=(20,8),color='green'
                                                                                       ,edgecolor='black',title='Fake')
plt.show()

#NGram Model 

##Generating N gram model

In [ ]:
data=' '.join([sentance for sentance in df['clean_text']])

In [ ]:
import nltk
from nltk.util import ngrams
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]
 
unigrams=extract_ngrams(data, 1)
bigrams= extract_ngrams(data, 2)
trigrams= extract_ngrams(data, 3)
fourgrams=extract_ngrams(data, 4)

##Counting the frequency of Ngram


In [ ]:
freq_uni = nltk.FreqDist(unigrams)
freq_bi = nltk.FreqDist(bigrams)
freq_tri = nltk.FreqDist(trigrams)
freq_four = nltk.FreqDist(fourgrams)

##Top 20 Unigram 

In [ ]:
# top 20 uigrams
top_20_uni=freq_uni.most_common(20)
top_20_uni_words,top_20_uni_freq=list(zip(*top_20_uni))
plt.figure(figsize=(20,7))
plt.bar(top_20_uni_words, top_20_uni_freq, color ='maroon',
        width = 0.4)

##Top 20 Bigram

In [ ]:
top_20_bi=freq_bi.most_common(20)
top_20_bi_words,top_20_bi_freq=list(zip(*top_20_bi))
plt.figure(figsize=(20,7))
plt.bar(top_20_bi_words, top_20_bi_freq, color ='lightcoral',
        width = 0.4)
plt.xticks(rotation=90) 

## Top 20 trigram

In [ ]:
top_20_tri=freq_tri.most_common(20)
top_20_tri_words,top_20_tri_freq=list(zip(*top_20_tri))
plt.figure(figsize=(20,7))
plt.bar(top_20_tri_words, top_20_tri_freq, color ='lightgreen',
        width = 0.4)
plt.xticks(rotation=90)

##Top 20 fourthgram

In [ ]:
top_20_four=freq_four.most_common(20)
top_20_four_words,top_20_four_freq=list(zip(*top_20_four))
plt.figure(figsize=(20,7))
plt.bar(top_20_four_words, top_20_four_freq, color ='blue',
        width = 0.4)
plt.xticks(rotation=90) 